In [1]:
# Import python packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import theano as T
import theano.tensor as tt
from pymc3.backends import SQLite
import seaborn as sns
import scipy as sp
import matplotlib as mp
import arviz as az
import pdb

# Helper functions
def indexall(L):
    poo = []
    for p in L:
        if not p in poo:
            poo.append(p)
    Ix = np.array([poo.index(p) for p in L])
    return poo,Ix

# Helper functions
def indexall_B(L,B):
    poo = []
    for p in L:
        if not p in poo:
            poo.append(p)
    Ix = np.array([poo.index(p) for p in L])
    a, b = poo.index(B), 0
    poo[b], poo[a] = poo[a], poo[b]
    
    Ix[Ix==b] = -1
    Ix[Ix==a] = 0
    Ix[Ix==-1] = a
    return poo,Ix

def subindexall(short,long):
    poo = []
    out = []
    for s,l in zip(short,long):
        if not l in poo:
            poo.append(l)
            out.append(s)
    return indexall(out)

match = lambda a, b: np.array([ b.index(x) if x in b else None for x in a ])
grep = lambda s, l: np.array([i for i in l if s in i])

# Function to standardize covariates
def stdize(x):
    return (x-np.mean(x))/(2*np.std(x))

# Coefficient of variation
cv =  lambda x: np.var(x) / np.mean(x)

In [2]:
# Import marine nutrient data and add traits

In [3]:
# Nutritional data
ndata = pd.read_excel('Nutrient_data.xlsx')
# Traits data
tdata = pd.read_excel('Traits_all.xlsx')

In [4]:

# Add traits information to nutritional dataframe
indx = match(ndata.Species_from_search.unique(),list(tdata.species_traits_all.values))
rindx = match(ndata.Species_from_search,list(ndata.Species_from_search.unique()))

# Traits to port over
tmp = ['Class', 'Order', 'Family','Genus_1', 'DemersPelag',
       'EnvTemp', 'DepthRangeDeep', 'trophic_level', 'Feeding_path', 'LMax',
       'BodyShapeI', 'K', 'tm']
# Port over
for trait in tmp:
    ndata[trait] = tdata[trait].values[indx][rindx]

In [5]:
ndata.columns

Index(['ID_no', 'Species_from_search', 'Protein_Replicates',
       'Protein_Value_perc', 'Protein_Variance', 'Zn_Replicates', 'Zn_Value',
       'Zn_Variance', 'Ca_Replicates', 'Ca_Value', 'Ca_Variance',
       'Fe_Replicates', 'Fe_Value', 'Fe_Variance', 'Se_Replicates', 'Se_Value',
       'Se_Variance', 'P_Replicates', 'P_Value', 'P_Variance', 'Mg_Replicates',
       'Mg_Value', 'Mg_Variance', 'VitA_Replicates', 'VitA_Value',
       'VitA_Variance', 'VitB12_Replicates', 'VitB12_Value', 'VitB12_Variance',
       'O3_Replicates', 'O3_Value', 'O3_Variance', 'Unnamed: 32', 'Form_Clean',
       'Search', 'Year', 'GeographicLocation_Cleaned', 'Family', 'Prep',
       'Protein_prep', 'Minerals_prep', 'Vitamin_prep', 'FA_prep',
       'Protein_Sample_Weight', 'Zn_Sample_Weight', 'Ca_Sample_Weight',
       'Fe_Sample_Weight', 'Se_Sample_Weight', 'P_Sample_Weight',
       'Mg_Sample_Weight', 'VitA_Sample_Weight', 'VitB12_Sample_Weight',
       'O3_Sample_Weight', 'Class', 'Order', 'Genus_1', '

In [6]:
# Correct names
ndata['DemersPelag'] = np.where(ndata.DemersPelag=='pelagic_oceanic', 'pelagic', ndata.DemersPelag)

# Remove space
ndata.Order = ndata.Order.str.replace(' ', '') 


In [7]:
# Import cleaned freshwater nutrients data
fdata = pd.read_csv('INFOODS_species_traits.csv')

In [8]:
fdata.columns

Index(['Unnamed: 0', 'source', 'year', 'geographic_location_cleaned', 'hdi',
       'demers_pelag', 'env_temp', 'form_clean', 'species_from_search',
       'family', 'paper_id', 'nutrient', 'value', 'SpecCode', 'Order',
       'DemersPelag', 'DepthRangeDeep', 'BodyShapeI', 'Lmax', 'EnvTemp', 'TL',
       'K', 'tm', 'Habitat', 'pelagic'],
      dtype='object')

In [9]:
fdata.columns = ['Unnamed: 0', 'Search', 'Year', 'GeographicLocation_Cleaned', 'hdi',
       'demers_pelag', 'env_temp', 'Form_Clean', 'Species_from_search',
       'Family', 'paper_id', 'nutrient', 'nutvalue', 'SpecCode', 'Order',
       'DemersPelag', 'DepthRangeDeep', 'BodyShapeI', 'LMax', 'EnvTemp', 'trophic_level',
       'K', 'tm', 'Habitat', 'pelagic']

In [10]:
np.unique(fdata.nutrient)

array(['Calcium', 'Iron', 'Omega-3', 'Protein', 'Selenium', 'Vitamin A',
       'Zinc'], dtype=object)

In [11]:
# Correct names
fdata['DemersPelag'] = np.where(fdata.DemersPelag=='pelagic-neritic', 'pelagic_neritic', fdata.DemersPelag)
fdata.Search = fdata.Search.str.replace('WoS', 'WOS')
fdata.Search = fdata.Search.str.replace('FAO/InFoods', 'FAO')

# Add underscore
fdata.Species_from_search = ndata.Species_from_search.str.replace(' ', '_')


In [35]:
# Drop rows where species are nan
fdata = fdata[fdata['Species_from_search'].notna()]

# Tidy up marine data

In [36]:
tmp = ndata[['Search', 'Year', 'GeographicLocation_Cleaned','Form_Clean', 'Species_from_search',
       'Family', 'Order', 'DemersPelag', 'DepthRangeDeep', 'BodyShapeI', 'LMax', 'EnvTemp', 'trophic_level',
       'K', 'tm', 'Ca_Value', 'Fe_Value', 'O3_Value', 'Protein_Value_perc', 'Se_Value', 'VitA_Value', 'Zn_Value']].copy()

Grab nutrient for each column

In [37]:
nutz = ['Ca_Value', 'Fe_Value', 'O3_Value', 'Protein_Value_perc', 'Se_Value', 'VitA_Value', 'Zn_Value']
nuts = ['Calcium', 'Iron', 'Omega-3', 'Protein', 'Selenium', 'Vitamin_A','Zinc']
valx = nutz[0]
poo = tmp.loc[np.isfinite(tmp[valx]),['Search', 'Year', 'GeographicLocation_Cleaned', 'DemersPelag', 'Form_Clean', 'Species_from_search',
       'Family', 'Order', 'DepthRangeDeep', 'BodyShapeI', 'LMax', 'EnvTemp', 'trophic_level','K', 'tm', valx] ].copy()

poo['nutrient'] = nuts[0]
poo['nutvalue'] = poo[valx]

mdata = poo.drop(columns=[valx]).copy()

In [38]:
for i in range(1,len(nutz)):
    valx = nutz[i]
    poo = tmp.loc[np.isfinite(tmp[valx]),['Search', 'Year', 'GeographicLocation_Cleaned', 'DemersPelag', 'Form_Clean', 'Species_from_search',
       'Family', 'Order', 'DepthRangeDeep', 'BodyShapeI', 'LMax', 'EnvTemp', 'trophic_level','K', 'tm', valx] ].copy()
    poo['nutrient'] = nuts[i]
    poo['nutvalue'] = poo[valx]
    mdata = mdata.append(poo.drop(columns=[valx]))
    
mdata['Habitat'] = 'Marine'

In [39]:
fdata.columns

Index(['Unnamed: 0', 'Search', 'Year', 'GeographicLocation_Cleaned', 'hdi',
       'demers_pelag', 'env_temp', 'Form_Clean', 'Species_from_search',
       'Family', 'paper_id', 'nutrient', 'nutvalue', 'SpecCode', 'Order',
       'DemersPelag', 'DepthRangeDeep', 'BodyShapeI', 'LMax', 'EnvTemp',
       'trophic_level', 'K', 'tm', 'Habitat', 'pelagic'],
      dtype='object')

# Merge datasets

In [40]:
out = mdata.append(fdata[mdata.columns]).copy()

In [41]:
out.head()

,Search,Year,GeographicLocation_Cleaned,DemersPelag,Form_Clean,Species_from_search,Family,Order,DepthRangeDeep,BodyShapeI,LMax,EnvTemp,trophic_level,K,tm,nutrient,nutvalue,Habitat
109,WOS,2009.0,Portugal,pelagic,muscle,Aphanopus_carbo,Trichiuridae,Perciformes,1700.0,elongate,115.4,polar_deep,4.500000,0.11,5.331634,Calcium,14.0,Marine
110,WOS,2015.0,Bangladesh,benthopelagic,whole_parts,Pampus_argenteus,Stromateidae,Perciformes,110.0,short_deep,60.0,subtropical,3.300000,0.45,1.631748,Calcium,31.0,Marine
111,WOS,2015.0,Bangladesh,pelagic_neritic,whole_parts,Stolephorus_tri,Engraulidae,Clupeiformes,50.0,fusiform,9.5,tropical,3.233333,1.03,0.800000,Calcium,1500.0,Marine
112,WOS,2015.0,Bangladesh,demersal,whole_parts,Johnius_belangerii,Sciaenidae,Perciformes,40.0,fusiform,30.0,tropical,3.270000,0.57,1.300000,Calcium,1900.0,Marine
113,WOS,2015.0,Bangladesh,pelagic_neritic,whole_parts,Scomberomorus_guttatus,Scombridae,Perciformes,200.0,fusiform,76.0,tropical,4.300000,0.34,1.900000,Calcium,34.0,Marine


In [42]:
out.columns.values

array(['Search', 'Year', 'GeographicLocation_Cleaned', 'DemersPelag',
       'Form_Clean', 'Species_from_search', 'Family', 'Order',
       'DepthRangeDeep', 'BodyShapeI', 'LMax', 'EnvTemp', 'trophic_level',
       'K', 'tm', 'nutrient', 'nutvalue', 'Habitat'], dtype=object)

In [43]:
out.DemersPelag.unique()

array(['pelagic', 'benthopelagic', 'pelagic_neritic', 'demersal',
       'reef_associated'], dtype=object)

In [44]:
np.sort(out.Order.unique())

array(['Acipenseriformes', 'Anguilliformes', 'Atheriniformes',
       'Aulopiformes', 'Beloniformes', 'Carcharhiniformes',
       'Chimaeriformes', 'Clupeiformes', 'Cypriniformes', 'Elopiformes',
       'Esociformes', 'Gadiformes', 'Lamniformes', 'Lampriformes',
       'Lophiiformes', 'Mugiliformes', 'Myctophiformes',
       'Myliobatiformes', 'Ophidiiformes', 'Osmeriformes',
       'Osteoglossiformes', 'Perciformes', 'Pleuronectiformes',
       'Rajiformes', 'Salmoniformes', 'Scorpaeniformes', 'Siluriformes',
       'Squaliformes', 'Synbranchiformes', 'Tetraodontiformes',
       'Torpediniformes', 'Zeiformes'], dtype=object)

In [45]:
np.sort(mdata.Species_from_search.unique())

array(['Acanthistius_brasilianus', 'Aetomylaeus_bovinus',
       'Aldrichetta_forsteri', 'Alepocephalus_agassizii',
       'Alepocephalus_bairdii', 'Alosa_alosa', 'Alosa_fallax',
       'Alosa_immaculata', 'Alosa_sapidissima', 'Ammodytes_hexapterus',
       'Anarhichas_lupus', 'Anchoa_hepsetus', 'Anguilla_anguilla',
       'Anguilla_japonica', 'Anguilla_rostrata', 'Aphanopus_carbo',
       'Argyrosomus_hololepidotus', 'Ariomma_bondi', 'Arius_maculatus',
       'Arripis_trutta', 'Atherina', 'Atherina_boyeri',
       'Balistes_capriscus', 'Bassanago_albescens',
       'Bathylagus_antarcticus', 'Bathyraja_brachyurops',
       'Bathyraja_macloviana', 'Bathyraja_scaphiops', 'Belone_belone',
       'Benthosema_pterotum', 'Beringraja_binoculata', 'Boops_boops',
       'Brama_brama', 'Brama_japonica', 'Callorhinchus_callorynchus',
       'Carangoides_orthogrammus', 'Caranx_hippos',
       'Carcharhinus_limbatus', 'Caulolatilus_intermedius',
       'Centrophorus_squamosus', 'Centropomus_paralle

In [46]:
np.sort(out.EnvTemp.unique())

array(['polar_deep', 'subtropical', 'temperate', 'tropical'], dtype=object)

In [47]:
out.Species_from_search[np.isnan(out.DepthRangeDeep)].unique()

array(['Merluccius_merluccius', 'Scomber_scombrus',
       'Merluccius_gayi_gayi', 'Engraulis_encrasicolus',
       'Prionace_glauca', 'Micromesistius_poutassou', 'Solea_solea',
       'Dicentrarchus_labrax', 'Sparus_aurata', 'Lepidorhombus_boscii',
       'Mullus_surmuletus', 'Thunnus_thynnus', 'Balistes_capriscus',
       'Galeichthys_feliceps', 'Trichiurus_lepturus', 'Scomber_japonicus',
       'Micromesistius_australis', 'Discopyge_tschudii',
       'Zearaja_chilensis', 'Psammobatis_scobina', 'Psammobatis_normani',
       'Bathyraja_brachyurops', 'Bathyraja_macloviana',
       'Bathyraja_scaphiops', 'Callorhinchus_callorynchus',
       'Bassanago_albescens', 'Salilota_australis',
       'Notophycis_marginata', 'Macruronus_magellanicus',
       'Merluccius_australis', 'Merluccius_hubbsi',
       'Coelorinchus_fasciatus', 'Genypterus_blacodes',
       'Sebastes_oculatus', 'Prionotus_nudigula',
       'Congiopodus_peruvianus', 'Acanthistius_brasilianus',
       'Brama_brama', 'Iluocoe

In [48]:
np.sort(out.DepthRangeDeep.unique())

array([1.00000000e+00, 2.00000000e+00, 5.00000000e+00, 6.00000000e+00,
       7.00000000e+00, 9.46600000e+00, 9.46666667e+00, 9.66666667e+00,
       1.00000000e+01, 1.20000000e+01, 1.30000000e+01, 1.60000000e+01,
       1.80000000e+01, 2.00000000e+01, 2.20000000e+01, 2.30000000e+01,
       2.50000000e+01, 2.70000000e+01, 3.00000000e+01, 3.90000000e+01,
       4.00000000e+01, 4.73846154e+01, 5.00000000e+01, 5.16666667e+01,
       5.30000000e+01, 5.60000000e+01, 6.00000000e+01, 7.00000000e+01,
       7.26666667e+01, 7.51157895e+01, 7.80000000e+01, 8.00000000e+01,
       8.11111111e+01, 9.00000000e+01, 9.50000000e+01, 9.85333333e+01,
       1.00000000e+02, 1.05000000e+02, 1.07058824e+02, 1.10000000e+02,
       1.10882353e+02, 1.15000000e+02, 1.16000000e+02, 1.18166667e+02,
       1.20000000e+02, 1.21000000e+02, 1.25000000e+02, 1.28000000e+02,
       1.33000000e+02, 1.47212121e+02, 1.50000000e+02, 1.53500000e+02,
       1.58141026e+02, 1.65000000e+02, 1.68000000e+02, 1.70000000e+02,
      

In [49]:
np.sort(out.LMax.unique())

array([  4.1       ,   5.3       ,   7.        ,   7.3       ,
         8.        ,   8.19999981,   9.        ,   9.5       ,
         9.6       ,  10.        ,  11.        ,  12.        ,
        12.3       ,  12.5       ,  12.52      ,  13.        ,
        13.19999981,  14.        ,  14.5       ,  15.        ,
        15.1       ,  15.30000019,  15.32500017,  15.8       ,
        16.        ,  16.1       ,  17.        ,  17.03809525,
        17.87      ,  18.        ,  19.        ,  20.        ,
        21.        ,  21.66      ,  22.        ,  22.1       ,
        22.60000038,  23.        ,  24.        ,  24.7750001 ,
        25.        ,  26.        ,  27.        ,  27.5       ,
        28.        ,  28.76      ,  30.        ,  30.5       ,
        30.95749995,  31.        ,  32.        ,  32.2542    ,
        33.        ,  33.2       ,  34.        ,  34.5       ,
        35.        ,  35.58000183,  35.6       ,  36.        ,
        37.        ,  38.        ,  38.09      ,  38.2 

In [50]:
np.sort(out.BodyShapeI.unique())

array(['elongate', 'flat', 'fusiform', 'short_deep'], dtype=object)

In [51]:
np.sort(out.K.unique())

array([0.02      , 0.03      , 0.039     , 0.04      , 0.05      ,
       0.06      , 0.062     , 0.0665    , 0.07      , 0.071     ,
       0.08      , 0.08      , 0.081     , 0.084     , 0.08491667,
       0.085     , 0.08528571, 0.087     , 0.09      , 0.09      ,
       0.091     , 0.093     , 0.096     , 0.1       , 0.1       ,
       0.101     , 0.1035    , 0.10721595, 0.109     , 0.109     ,
       0.11      , 0.111     , 0.113     , 0.116     , 0.117     ,
       0.118     , 0.119     , 0.12      , 0.12      , 0.121     ,
       0.1225    , 0.123     , 0.124     , 0.125     , 0.13      ,
       0.13      , 0.13044444, 0.131     , 0.13353333, 0.13500001,
       0.13600001, 0.137     , 0.1374125 , 0.14      , 0.14      ,
       0.14399999, 0.145     , 0.148     , 0.15      , 0.15000001,
       0.15099999, 0.15199999, 0.15433333, 0.157     , 0.16      ,
       0.16      , 0.162     , 0.163     , 0.16400001, 0.16472857,
       0.16566667, 0.17      , 0.17      , 0.17033333, 0.171  

In [52]:
np.sort(out.tm.unique())

array([ 0.4       ,  0.5       ,  0.5008605 ,  0.5107421 ,  0.53498147,
        0.56868269,  0.6       ,  0.62172303,  0.6922428 ,  0.7       ,
        0.772142  ,  0.7939364 ,  0.8       ,  0.8015776 ,  0.82793668,
        0.87723556,  0.9       ,  0.90370642,  0.9204233 ,  0.9337703 ,
        0.9414889 ,  0.9472714 ,  0.95000548,  0.99900434,  1.        ,
        1.00236669,  1.010677  ,  1.02122152,  1.022433  ,  1.08703613,
        1.1       ,  1.13326914,  1.142547  ,  1.19487854,  1.22413702,
        1.23149135,  1.3       ,  1.30164839,  1.3073505 ,  1.316123  ,
        1.32164794,  1.34480006,  1.37678   ,  1.384019  ,  1.38963222,
        1.390072  ,  1.4       ,  1.40532   ,  1.40885   ,  1.411942  ,
        1.418672  ,  1.41939248,  1.42652644,  1.43524   ,  1.5       ,
        1.515498  ,  1.52996036,  1.53045   ,  1.553669  ,  1.55741994,
        1.558404  ,  1.571342  ,  1.59151   ,  1.6       ,  1.60349842,
        1.6133444 ,  1.631748  ,  1.640862  ,  1.66429   ,  1.68

In [53]:
np.sort(out.Search.unique())

array(['Expert', 'FAO', 'WOS'], dtype=object)

In [54]:
np.sort(out.Form_Clean.unique())

array(['muscle', 'not specified', 'unknown', 'whole', 'whole_parts'],
      dtype=object)

In [55]:
# np.sort(fdata.Prep.unique())

In [56]:
out.to_csv('MeasuredNutrients.csv')

In [57]:
fdata.Species_from_search.unique()

array(['Merluccius_merluccius', 'Scomber_scombrus',
       'Merluccius_gayi_gayi', 'Engraulis_encrasicolus',
       'Lophius_piscatorius', 'Prionace_glauca',
       'Micromesistius_poutassou', 'Galeus_melastomus', 'Solea_solea',
       'Dicentrarchus_labrax', 'Sparus_aurata', 'Lepidorhombus_boscii',
       'Mullus_surmuletus', 'Salmo_salar', 'Sardina_pilchardus',
       'Trachurus_trachurus', 'Xiphias_gladius', 'Thunnus_thynnus',
       'Balistes_capriscus', 'Clupea_harengus', 'Galeichthys_feliceps',
       'Trichiurus_lepturus', 'Scomber_japonicus',
       'Micromesistius_australis', 'Discopyge_tschudii',
       'Zearaja_chilensis', 'Psammobatis_scobina', 'Psammobatis_normani',
       'Bathyraja_brachyurops', 'Bathyraja_macloviana',
       'Bathyraja_scaphiops', 'Callorhinchus_callorynchus',
       'Bassanago_albescens', 'Salilota_australis',
       'Notophycis_marginata', 'Macruronus_magellanicus',
       'Merluccius_australis', 'Merluccius_hubbsi',
       'Coelorinchus_fasciatus', '